In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark titanic") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.csv("titanic.csv", header=True, inferSchema=True)

In [4]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [5]:
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [6]:
c_df = df.groupBy(df.Age < 55).count()

total = c_df.select("count").agg({"count": "sum"}).collect().pop()["sum(count)"]
result = c_df.withColumn("percent (%)", (c_df["count"] / total) * 100)

result.show()

+----------+-----+------------------+
|(Age < 55)|count|       percent (%)|
+----------+-----+------------------+
|      null|  177|19.865319865319865|
|      true|  672| 75.42087542087542|
|     false|   42| 4.713804713804714|
+----------+-----+------------------+



In [7]:
df = df.na.fill(0)

In [8]:
df.groupBy((df.Age < 10),
                  (df.Age >= 10) & (df.Age <= 50),
                  (df.Age > 50)).count().show()

+----------+-----------------------------+----------+-----+
|(Age < 10)|((Age >= 10) AND (Age <= 50))|(Age > 50)|count|
+----------+-----------------------------+----------+-----+
|      true|                        false|     false|  239|
|     false|                         true|     false|  588|
|     false|                        false|      true|   64|
+----------+-----------------------------+----------+-----+



In [9]:
from pyspark.ml.feature import Bucketizer

bucketizer = Bucketizer(splits=[0, 10, 50, float("Inf") ],inputCol="Age", outputCol="buckets")
df_bucketed = bucketizer.setHandleInvalid("keep").transform(df)

df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

age_dict = {0.0: "CHILDREN", 1.0: "YOUNG", 2.0: "ADULT"}
titanic_udf = udf(lambda X: age_dict[X], StringType())
df_bucketed = df_bucketed.withColumn("age_group", titanic_udf("buckets"))

df_bucketed.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+---------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|buckets|age_group|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+---------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    1.0|    YOUNG|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    1.0|    YOUNG|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|    1.0|    YOUNG|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|    1.0|    YOUNG|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0| 

In [11]:
q2_df = df_bucketed.groupBy((df_bucketed.age_group == "CHILDREN"),
                            (df_bucketed.age_group == "YOUNG"),
                            (df_bucketed.age_group == "OLD"),
                           ).count()

total = c_df.select("count").agg({"count": "sum"}).collect().pop()["sum(count)"]
result = c_df.withColumn("percent (%)", (c_df["count"] / total) * 100)

result.show()

+----------+-----+------------------+
|(Age < 55)|count|       percent (%)|
+----------+-----+------------------+
|      null|  177|19.865319865319865|
|      true|  672| 75.42087542087542|
|     false|   42| 4.713804713804714|
+----------+-----+------------------+

